

---

******************************************* DESCRIZIONE DEL PROGETTO *******************************************

Il progetto è composto da tre parti principali: la creazione e popolazione di un database NOSQL avente più documenti, realizzato con MongoDB a partire da un dataset, l''analisi statistica del dataset, avente grafici realizzati utilizzando PowerBi, che viene connesso al database MongoDB tramite ODBC, ed una webapp istanziata su server Flask, che si connette al database utilizzando PyMongo per mostrare lo stato della connessione.  

La repository github che contiene il progetto è [github.com/FraPCA/WebFuel](https://github.com/FraPCA/WebFuel)

PROGETTAZIONE ED IMPLEMENTAZIONE DEL DATABASE

Si è scelto di utilizzare MongoDB a seguito di un''analisi di altre soluzioni NOSQL document-based, come RavenDB e TinyDB.
MongoDB è risultata l''unica opzione che supporta la connessione tramite ODBC a PowerBi, per realizzare i grafici nella parte di analisi statistica.

Una parte fondamentale della proposta di progetto realizzata è l''utilizzo delle funzionalità di replica set offerte da MongoDB per dimostrarne il funzionamento in un progetto tipico, e migliorare la disponibilità e scalabilità del database.
Per semplificare la realizzazione del replica set, si è deciso di installare tutti e 3 i nodi che lo compongono sullo stesso pc: in questo modo, risulta più chiara la latenza dovuta alle operazioni dei nodi, che sarebbe altrimenti confondibile con la latenza di rete.
Abbiamo creato 3 nodi in modo da poter dimostrare, con il minimo spreco di risorse computazionali, il comportamento del database in situazioni di fallimento non catastrofico con e senza maggioranza.

I nodi creati sono localhost:27017, localhost:27018 e localhost:27019, rispettivamente denominati NodeA, NodeB e NodeC.
Appartengono al replica set rs0.
Per inizializzare il replica set, si è utilizzato lo strumento MongoSH per inviare il comando rs.initiate().

Il replica set permette connessioni utilizzando connection string che fanno riferimento sia ad uno solo dei nodi, che a più di essi.
(es. localhost:27017,localhost:27018,localhost:27019/)

Abbiamo testato connessioni sia da client locali, come MongoSH e MongoDB Compass, che con PyMongo tramite Python.
Il database accetta connessioni se almeno due nodi sono attivi, uno primario ed uno secondario. Nel caso in cui un solo nodo è attivo, rifiuta le connessioni.

Per avviare i nodi del replica set, è necessario avviare per ognuno una nuova istanza di terminale (in quanto il processo è bloccante, se si vuole visualizzare il log d''esecuzione) attraverso il comando:
.\mongod --dbpath ../<nomeNodo> --port <numPortaNodo> --replSet "rs0"
Tale comando deve essere eseguito nella cartella di installazione di MongoDB, nella sottocartella Server\<nomeversione>\bin.

Il dataset viene correttamente replicato su tutti i nodi connessi: è stato testato che la connessione al database fornisce i dati corretti con ciascuno dei 3 nodi come nodo primario.





INTRODUZIONE E OBIETTIVI DELL’ANALISI STATISTICA

L’obiettivo è fare un’analisi descrittiva incentrata sull’impatto ambientale e sull’autonomia di automobili, facendo particolare attenzione a: tendenze e cambiamenti nel tempo, e alla diffusione e ai consumi dei carburanti più diffusi.

In particolare:

Fare quindi un’analisi sull’autonomia di veicoli automobilistici. Analizzare quali sono le marche che producono i veicoli con la maggiore autonomia, e in che modo il tipo di carburante utilizzato impatta l’autonomia.

Osservare quali sono i modelli che hanno un minore impatto ambientale e quali tipi di carburante sono meno inquinanti.

Individuare e osservare trend e cambiamenti nel tempo, dal punto di vista dell’efficienza di ogni tipo di carburante.

Stabilire se, al giorno d’oggi, le auto elettriche siano un’alternativa valida ai modelli termici, considerando sia vantaggi che costi.

Con queste informazioni, si sarà in grado di delineare un quadro chiaro e aggiornato del panorama attuale dell'efficienza del carburante nel settore automobilistico.



DESCRIZIONE DEL DATASET

Il dataset “Vehicle Fuel Economy”, disponibile su Kaggle al seguente [Link](https://www.kaggle.com/datasets/sahirmaharajj/fuel-economy/data), contiene le informazioni di 38113 entry. Ogni dato corrisponde a un modello di macchina univoco e diverso. Il dataset contiene 81 colonne, che danno informazioni su consumi, emissioni e caratteristiche fisiche di ogni veicolo. Sono stati ottenuti dati provenienti esclusivamente dagli USA. Tramite Data Cleaning, il numero di colonne è stato ridotto a 38. Il processo di data cleaning è eseguibile in questo notebook: prende come input il file fuel.csv e restituisce il file a seguito delle trasformazioni di cleaning, fuel_cleaned.csv

Le unità di misura imperiali sono state convertite al sistema metrico: Nel caso di miles per gallon, i dati corrispondenti sono stati trasformati in chilometri per litro.

Anche le informazioni economiche sono state convertite da dollari ad euro.

Le modifiche di conversione sono eseguibili con il notebook convert.ipynb, che prende come input il file fuel_cleaned.csv restituito da questo notebook e restituisce in output il file con le unità di misure metriche, fuel_converted.csv

DIVISIONE DEL DATASET

Il dataset convertito è stato poi diviso in 3 documenti, in modo da permettere la realizzazione di query su più documenti nel database e per organizzare in più aree tematiche i dati di interesse presenti.

Il processo di divisione viene eseguito con il notebook split.ipynb, che prende in input il file fuel_converted.csv e resituisce in output i tre file corrispondenti:

veicoli.csv, che contiene i dati utili all'identificazione del veicolo, tra cui l'anno di commercializzazione, il nome, la casa produttrice, la sua trasmissione ed il volume.

emissioni.csv, che contiene i dettagli del motore utilizzato nel veicolo, i tipi di carburante ed i punteggi relativi alle emissioni.

consumi.csv, che contiene le informazioni relative al consumo in autostrada ed in città, l'autonomia dimostrata ed il suo costo economico ed emissivo.

Il meccanismo utilizzato per la divisione dei dataset è stato quello delle references: dopo aver isolato le colonne desiderate, si è aggiunta la colonna vehicle_id corrispondente, che viene utilizzata per fare riferimento ai dati dello stesso veicolo nei documenti.

CONNESSIONE A POWERBI E REALIZZAZIONE DEI GRAFICI

Dopo aver installato MongoDB e Power BI sul proprio dispositivo, è necessario utilizzare un connettore affinché Power BI possa reperire il dataset da MongoDB. Per fare ciò, si può utilizzare la seguente guida: [Link](https://www.youtube.com/watch?v=2_uFlLYjKhk). 

Bisogna innanzitutto installare il connettore ODBC, e configurarlo correttamente utilizzando il DNS corretto per la connessione a MongoDB. Per connettere le due applicazioni, bisogna eseguire il programma mongosqld precedentemente scaricato. A questo punto, bisogna aprire Power BI, creare un nuovo file, e nella sezione in cui si sceglie la fonte dei dati da caricare, scegliere 'altro', e poi 'ODBC', e infine selezionare il connettore installato. A questo punto si possono selezionare i dataset che si vogliono importare da MongoDB a Power BI. 

Dopo la connessione avvenuta con successo, Power BI mette a disposizione una serie di tool per la realizzazione dei grafici. Basterà selezionare le colonne di interesse, e trascinarle nella sezione di 'visualizzazione dei dati' per ottenere dei grafici. Successivamente si potranno personalizzare le forme e i colori per dedicarsi all'estetica dei grafici.

PROGETTAZIONE ED IMPLEMENTAZIONE WEBAPP

Si è deciso di creare una webapp per visualizzare lo stato dei nodi del replica set, e mostrare gli effetti delle elezioni durante l'operazione normale del DBMS ed in caso di fallimento dei nodi.

L'idea originale di progetto era quella di una pagina organizzata come dashboard.

Il framework web che abbiamo scelto di usare è Flask, in quanto veloce da utilizzare e facilmente collegabile con Python.

La pagina web che fa da dashboard si trova all'indirizzo localhost:5000/status, dopo aver fatto partire il server con il comando flask run.


Per ottenere le informazioni sui nodi del replica set, si stabilisce innanzitutto una connessione PyMongo tra Flask ed il replica set. Successivamente, viene inviata una richiesta GET, e si riceve l'output pulito e riformattato del comando replSetGetStatus sul database.

Dall'output originale, prima di essere inviato al browser, si cancellano dei campi inutilizzati, e si convertono i tipi di dato TimeStamp creati da MongoDB in stringhe con formato D/M/Y, H/M/S/MS .

Il messaggio JSON ricevuto contiene informazioni sui nodi connessi, il loro stato, statistiche come la latenza, l'heartbeat ed eventuali errori, oltre che sulle elezioni e sulla maggioranza per lettura/scrittura.

Inizialmente il messaggio veniva stampato a schermo, successivamente si è pensato ad un'interfaccia dinamica che mostra visualmente lo stato dei 3 nodi appartenenti al cluster.

La parte grafica della webapp è stata realizzata con la libreria Tailwind, attraverso un misto di HTML e CSS.

Ognuno dei 3 nodi ha un'icona:


*   Grigia con croce rossa, il nodo risulta spento/non raggiungibile.
*   Verde con punto interrogativo blu, il nodo è raggiungibile ed attivo, ma non sono disponibili gli altri nodi per eleggere il nodo primario. Non è possibile connettersi al DB.
*   Verde con corona gialla, il nodo è attivo ed è stato eletto Primario.
*   Verde senza corona, il nodo è attivo ed è stato eletto Secondario.
*   Verde con simbolo dell'accensione arancione, il nodo ha ricevuto il comando di Shutdown e sta eseguendo le operazioni finali prima di spegnersi.

Se almeno due nodi sono verdi, abbiamo una connessione stabile con il database.

Dopo aver provato la corretta gestione dello Status, e delle icone nella pagina, c'era bisogno di rendere dinamica la pagina, per mostrare gli aggiornamenti dei nodi senza effettuare refresh dal browser. La soluzione implementata utilizza la libreria SocketIO per inviare, attraverso una funzione Javascript, la richiesta GET per ottenere lo Status attraverso un socket che connette il server Flask al browser.

Utilizzando i socket, si evita di sovraccaricare il server, che invece gestisce un canale di comunicazione con basso overhead, invia frequenti aggiornamenti (una volta ogni cinque secondi) e gestisce automaticamente la riconnessione.

Un'altra funzione javascript aggiorna l'aspetto della pagina a seguito dell'avvenuta ricezione del messaggio di Status.

Abbiamo poi aggiunto del testo per indicare le informazioni ricevute su ciascuno dei nodi e sulle elezioni, che compare solo dopo aver stabilito una connessione con il database, e scompare quando essa termina.


Al termine dell'implementazione della gestione dello Status regolare (con due nodi verdi), abbiamo creato un'ulteriore funzione per gestire i messaggi di errore inviati da MongoDB quando non è possibile stabilire una connessione con il DB, che sono diversi a seconda dei casi:


*   Almeno un nodo attivo, incapace di stabilire maggioranza
*   Nessun nodo attivo, a causa di spegnimento o quiescenza

Per entrambi, l'eccezione lanciata da PyMongo viene gestita e viene creato un messaggio di errore corrispondente, contenente lo stato dei nodi, che viene visualizzato poi nella pagina web.







In [ ]:
import pandas as pd


In [ ]:
data = pd.read_csv("fuel.csv")

<ipython-input-2-b3a57def2abe>:1: DtypeWarning: Columns (7,44) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("fuel.csv")


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38113 entries, 0 to 38112
Data columns (total 81 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   vehicle_id                         38113 non-null  int64  
 1   year                               38113 non-null  int64  
 2   make                               38113 non-null  object 
 3   model                              38113 non-null  object 
 4   class                              38113 non-null  object 
 5   drive                              36924 non-null  object 
 6   transmission                       38102 non-null  object 
 7   transmission_type                  15045 non-null  object 
 8   engine_index                       38113 non-null  int64  
 9   engine_descriptor                  22693 non-null  object 
 10  engine_cylinders                   37977 non-null  float64
 11  engine_displacement                37979 non-null  flo

In [ ]:
data = data.dropna(axis=1,how='all') # Cancellazione delle colonne nulle

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38113 entries, 0 to 38112
Data columns (total 73 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   vehicle_id                         38113 non-null  int64  
 1   year                               38113 non-null  int64  
 2   make                               38113 non-null  object 
 3   model                              38113 non-null  object 
 4   class                              38113 non-null  object 
 5   drive                              36924 non-null  object 
 6   transmission                       38102 non-null  object 
 7   transmission_type                  15045 non-null  object 
 8   engine_index                       38113 non-null  int64  
 9   engine_descriptor                  22693 non-null  object 
 10  engine_cylinders                   37977 non-null  float64
 11  engine_displacement                37979 non-null  flo

In [ ]:
data.columns[data.isnull().any()] #Colonne con valori nulli

Index(['drive', 'transmission', 'transmission_type', 'engine_descriptor',
       'engine_cylinders', 'engine_displacement', 'turbocharger',
       'gas_guzzler_tax', 'alternative_fuel_technology'],
      dtype='object')

In [ ]:
data = data.drop('gas_guzzler_tax', axis=1) #Tassa americana, non serve


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38113 entries, 0 to 38112
Data columns (total 72 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   vehicle_id                         38113 non-null  int64  
 1   year                               38113 non-null  int64  
 2   make                               38113 non-null  object 
 3   model                              38113 non-null  object 
 4   class                              38113 non-null  object 
 5   drive                              36924 non-null  object 
 6   transmission                       38102 non-null  object 
 7   transmission_type                  15045 non-null  object 
 8   engine_index                       38113 non-null  int64  
 9   engine_descriptor                  22693 non-null  object 
 10  engine_cylinders                   37977 non-null  float64
 11  engine_displacement                37979 non-null  flo

In [ ]:
data.convert_dtypes() #Regolarizza tutti i NAN nel formato più appropriato al resto della colonna (0, ' ', 0.0)

,vehicle_id,year,make,model,class,drive,transmission,transmission_type,engine_index,engine_descriptor,...,hours_to_charge_240v,hours_to_charge_ac_240v,composite_city_mpg,composite_highway_mpg,composite_combined_mpg,range_ft1,city_range_ft1,highway_range_ft1,city_range_ft2,highway_range_ft2
0,26587,1984,Alfa Romeo,GT V6 2.5,Minicompact Cars,<NA>,Manual 5-Speed,<NA>,9001,(FFS),...,0.0,0.0,0,0,0,0,0.0,0.0,0.0,0.0
1,27705,1984,Alfa Romeo,GT V6 2.5,Minicompact Cars,<NA>,Manual 5-Speed,<NA>,9005,(FFS) CA model,...,0.0,0.0,0,0,0,0,0.0,0.0,0.0,0.0
2,26561,1984,Alfa Romeo,Spider Veloce 2000,Two Seaters,<NA>,Manual 5-Speed,<NA>,9002,(FFS),...,0.0,0.0,0,0,0,0,0.0,0.0,0.0,0.0
3,27681,1984,Alfa Romeo,Spider Veloce 2000,Two Seaters,<NA>,Manual 5-Speed,<NA>,9006,(FFS) CA model,...,0.0,0.0,0,0,0,0,0.0,0.0,0.0,0.0
4,27550,1984,AM General,DJ Po Vehicle 2WD,Special Purpose Vehicle 2WD,2-Wheel Drive,Automatic 3-Speed,<NA>,1830,(FFS),...,0.0,0.0,0,0,0,0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38108,37564,2017,Volvo,XC60 FWD,Small Sport Utility Vehicle 2WD,Front-Wheel Drive,Automatic (S8),<NA>,90,SIDI,...,0.0,0.0,0,0,0,0,0.0,0.0,0.0,0.0
38109,37547,2017,Volvo,XC90 AWD,Standard Sport Utility Vehicle 4WD,All-Wheel Drive,Automatic (S8),<NA>,52,SIDI,...,0.0,0.0,0,0,0,0,0.0,0.0,0.0,0.0
38110,37548,2017,Volvo,XC90 AWD,Standard Sport Utility Vehicle 4WD,All-Wheel Drive,Automatic (S8),<NA>,53,SIDI,...,0.0,0.0,0,0,0,0,0.0,0.0,0.0,0.0
38111,37703,2017,Volvo,XC90 AWD PHEV,Standard Sport Utility Vehicle 4WD,All-Wheel Drive,Automatic (S8),<NA>,54,SIDI; PHEV,...,3.0,0.0,29,32,30,0,0.0,0.0,13.84,13.3


In [ ]:
data.info() #Finale

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38113 entries, 0 to 38112
Data columns (total 72 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   vehicle_id                         38113 non-null  int64  
 1   year                               38113 non-null  int64  
 2   make                               38113 non-null  object 
 3   model                              38113 non-null  object 
 4   class                              38113 non-null  object 
 5   drive                              36924 non-null  object 
 6   transmission                       38102 non-null  object 
 7   transmission_type                  15045 non-null  object 
 8   engine_index                       38113 non-null  int64  
 9   engine_descriptor                  22693 non-null  object 
 10  engine_cylinders                   37977 non-null  float64
 11  engine_displacement                37979 non-null  flo

In [ ]:
data.drop(list(data.filter(regex = 'unrounded')), axis = 1, inplace = True) #Valori unrounded non sono utilizzati, in quanto mancano per la maggioranza delle entry ed il livello di approssimazione dei dati rounded è accettabile.


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38113 entries, 0 to 38112
Data columns (total 66 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   vehicle_id                         38113 non-null  int64  
 1   year                               38113 non-null  int64  
 2   make                               38113 non-null  object 
 3   model                              38113 non-null  object 
 4   class                              38113 non-null  object 
 5   drive                              36924 non-null  object 
 6   transmission                       38102 non-null  object 
 7   transmission_type                  15045 non-null  object 
 8   engine_index                       38113 non-null  int64  
 9   engine_descriptor                  22693 non-null  object 
 10  engine_cylinders                   37977 non-null  float64
 11  engine_displacement                37979 non-null  flo

In [ ]:
data.astype(bool).sum(axis=0)



vehicle_id           38113
year                 38113
make                 38113
model                38113
class                38113
                     ...  
range_ft1              133
city_range_ft1         106
highway_range_ft1      106
city_range_ft2          65
highway_range_ft2       65
Length: 66, dtype: int64

In [ ]:
data.drop(list(data.filter(regex = 'range')), axis = 1, inplace = True) #Valori range non sono utilizzati, in quanto mancano per la maggioranza delle entry ed il tipo di dato non è correlato all'analisi

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38113 entries, 0 to 38112
Data columns (total 61 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   vehicle_id                         38113 non-null  int64  
 1   year                               38113 non-null  int64  
 2   make                               38113 non-null  object 
 3   model                              38113 non-null  object 
 4   class                              38113 non-null  object 
 5   drive                              36924 non-null  object 
 6   transmission                       38102 non-null  object 
 7   transmission_type                  15045 non-null  object 
 8   engine_index                       38113 non-null  int64  
 9   engine_descriptor                  22693 non-null  object 
 10  engine_cylinders                   37977 non-null  float64
 11  engine_displacement                37979 non-null  flo

In [ ]:
data.astype(bool).sum(axis=0)


vehicle_id                 38113
year                       38113
make                       38113
model                      38113
class                      38113
                           ...  
hours_to_charge_240v         175
hours_to_charge_ac_240v       46
composite_city_mpg            65
composite_highway_mpg         65
composite_combined_mpg        65
Length: 61, dtype: int64

In [ ]:
data.drop(list(data.filter(regex = 'ft2')), axis = 1, inplace = True) #Rimozione dei valori relativi al Fuel Type 2, in quanto la colonna corrispondente aveva solamente valori nulli ed è stata eliminata.

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38113 entries, 0 to 38112
Data columns (total 52 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   vehicle_id                         38113 non-null  int64  
 1   year                               38113 non-null  int64  
 2   make                               38113 non-null  object 
 3   model                              38113 non-null  object 
 4   class                              38113 non-null  object 
 5   drive                              36924 non-null  object 
 6   transmission                       38102 non-null  object 
 7   transmission_type                  15045 non-null  object 
 8   engine_index                       38113 non-null  int64  
 9   engine_descriptor                  22693 non-null  object 
 10  engine_cylinders                   37977 non-null  float64
 11  engine_displacement                37979 non-null  flo

In [ ]:
data.astype(bool).sum(axis=0)


vehicle_id                           38113
year                                 38113
make                                 38113
model                                38113
class                                38113
drive                                38113
transmission                         38113
transmission_type                    38113
engine_index                         25517
engine_descriptor                    38113
engine_cylinders                     38113
engine_displacement                  38112
turbocharger                         38113
fuel_type                            38113
fuel_type_1                          38113
city_mpg_ft1                         38113
city_gasoline_consumption_cd            14
city_electricity_consumption           198
city_utility_factor                     65
highway_mpg_ft1                      38113
highway_gasoline_consumption_cd         11
highway_electricity_consumption        198
highway_utility_factor                  65
unadjusted_

In [ ]:
data.drop(list(data.filter(regex = '_cd')), axis = 1, inplace = True) #Rimozione dei valori relativi al consumo in modalità di esaurimento della carica per veicoli elettrici, in quanto la colonna corrispondente aveva un numero insufficiente di valori non-nulli.

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38113 entries, 0 to 38112
Data columns (total 48 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   vehicle_id                         38113 non-null  int64  
 1   year                               38113 non-null  int64  
 2   make                               38113 non-null  object 
 3   model                              38113 non-null  object 
 4   class                              38113 non-null  object 
 5   drive                              36924 non-null  object 
 6   transmission                       38102 non-null  object 
 7   transmission_type                  15045 non-null  object 
 8   engine_index                       38113 non-null  int64  
 9   engine_descriptor                  22693 non-null  object 
 10  engine_cylinders                   37977 non-null  float64
 11  engine_displacement                37979 non-null  flo

In [ ]:
data.astype(bool).sum(axis=0)


vehicle_id                           38113
year                                 38113
make                                 38113
model                                38113
class                                38113
drive                                38113
transmission                         38113
transmission_type                    38113
engine_index                         25517
engine_descriptor                    38113
engine_cylinders                     38113
engine_displacement                  38112
turbocharger                         38113
fuel_type                            38113
fuel_type_1                          38113
city_mpg_ft1                         38113
city_electricity_consumption           198
city_utility_factor                     65
highway_mpg_ft1                      38113
highway_electricity_consumption        198
highway_utility_factor                  65
unadjusted_city_mpg_ft1              38088
unadjusted_highway_mpg_ft1           38088
combined_mp

In [ ]:
data.drop(list(data.filter(regex = 'hours_to_charge')), axis = 1, inplace = True) #Rimozione dei valori relativi alle ore di ricarica dei veicoli ibridi/elettrici, in quanto non rilevanti all'analisi da effettuare.

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38113 entries, 0 to 38112
Data columns (total 45 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   vehicle_id                         38113 non-null  int64  
 1   year                               38113 non-null  int64  
 2   make                               38113 non-null  object 
 3   model                              38113 non-null  object 
 4   class                              38113 non-null  object 
 5   drive                              36924 non-null  object 
 6   transmission                       38102 non-null  object 
 7   transmission_type                  15045 non-null  object 
 8   engine_index                       38113 non-null  int64  
 9   engine_descriptor                  22693 non-null  object 
 10  engine_cylinders                   37977 non-null  float64
 11  engine_displacement                37979 non-null  flo

In [ ]:
data.drop(list(data.filter(regex = 'composite')), axis = 1, inplace = True) #Rimozione dei valori relativi al consumo composto per veicoli ibridi, in quanto aventi numero insufficiente di valori non nulli.

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38113 entries, 0 to 38112
Data columns (total 42 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   vehicle_id                         38113 non-null  int64  
 1   year                               38113 non-null  int64  
 2   make                               38113 non-null  object 
 3   model                              38113 non-null  object 
 4   class                              38113 non-null  object 
 5   drive                              36924 non-null  object 
 6   transmission                       38102 non-null  object 
 7   transmission_type                  15045 non-null  object 
 8   engine_index                       38113 non-null  int64  
 9   engine_descriptor                  22693 non-null  object 
 10  engine_cylinders                   37977 non-null  float64
 11  engine_displacement                37979 non-null  flo

In [ ]:
data.astype(bool).sum(axis=0)


vehicle_id                           38113
year                                 38113
make                                 38113
model                                38113
class                                38113
drive                                38113
transmission                         38113
transmission_type                    38113
engine_index                         25517
engine_descriptor                    38113
engine_cylinders                     38113
engine_displacement                  38112
turbocharger                         38113
fuel_type                            38113
fuel_type_1                          38113
city_mpg_ft1                         38113
city_electricity_consumption           198
city_utility_factor                     65
highway_mpg_ft1                      38113
highway_electricity_consumption        198
highway_utility_factor                  65
unadjusted_city_mpg_ft1              38088
unadjusted_highway_mpg_ft1           38088
combined_mp

In [ ]:
data.drop(list(data.filter(regex = '_utility_factor')), axis = 1, inplace = True) #Rimozione dei valori relativi al rapporto tra miglia viaggiate in modalità elettrica e miglia totali, in quanto aventi numero insufficiente di valori non nulli e non pertinente all'analisi.

In [ ]:
data.astype(bool).sum(axis = 0)

vehicle_id                           38113
year                                 38113
make                                 38113
model                                38113
class                                38113
drive                                38113
transmission                         38113
transmission_type                    38113
engine_index                         25517
engine_descriptor                    38113
engine_cylinders                     38113
engine_displacement                  38112
turbocharger                         38113
fuel_type                            38113
fuel_type_1                          38113
city_mpg_ft1                         38113
city_electricity_consumption           198
highway_mpg_ft1                      38113
highway_electricity_consumption        198
unadjusted_city_mpg_ft1              38088
unadjusted_highway_mpg_ft1           38088
combined_mpg_ft1                     38113
combined_electricity_consumption       198
annual_fuel

In [ ]:
data.transmission.value_counts()

transmission
Automatic 4-Speed                   11042
Manual 5-Speed                       8323
Automatic 3-Speed                    3151
Automatic (S6)                       2684
Manual 6-Speed                       2448
Automatic 5-Speed                    2191
Manual 4-Speed                       1483
Automatic 6-Speed                    1447
Automatic (S8)                        977
Automatic (S5)                        827
Automatic (variable gear ratios)      702
Automatic 7-Speed                     675
Automatic (S7)                        270
Auto(AM-S7)                           266
Automatic 8-Speed                     259
Automatic (S4)                        233
Auto(AM7)                             166
Auto(AV-S6)                           153
Automatic (A1)                        124
Auto(AM6)                             120
Automatic 9-Speed                     105
Auto(AM-S6)                            87
Auto(AV-S7)                            80
Manual 3-Speed       

In [ ]:
data.transmission = data.transmission.replace('Manual 5 Speed', 'Manual 5-Speed')
data.transmission = data.transmission.replace('Automatic (S5)', 'Automatic 5-Speed')
data.transmission = data.transmission.replace('Automatic (S4)', 'Automatic 4-Speed')
data.transmission = data.transmission.replace('Automatic (S6)', 'Automatic 6-Speed')

In [ ]:
data.transmission.unique()

array(['Manual 5-Speed', 'Automatic 3-Speed', 'Manual 4-Speed',
       'Automatic 4-Speed', 'Manual 3-Speed', 'Manual 4-Speed Doubled',
       nan, 'Manual 6-Speed', 'Automatic (variable gear ratios)',
       'Automatic 5-Speed', 'Auto(L4)', 'Auto(L3)', 'Automatic (A1)',
       'Automatic 6-Speed', 'Automatic 7-Speed', 'Automatic (S7)',
       'Automatic (S8)', 'Automatic (AV)', 'Auto(AM7)', 'Automatic (AM5)',
       'Auto(AM6)', 'Automatic (AM6)', 'Automatic (A6)', 'Auto(AV-S7)',
       'Manual 7-Speed', 'Auto (AV-S6)', 'Auto (AV-S8)',
       'Automatic (AV-S6)', 'Auto(AV-S6)', 'Auto(AM5)', 'Auto (AV)',
       'Auto(AV-S8)', 'Automatic 8-Speed', 'Auto(AM-S6)', 'Auto(AM-S7)',
       'Automatic 6Speed', 'Automatic 9-Speed', 'Automatic (S9)',
       'Auto(AM-S8)', 'Auto(AM8)', 'Manual(M7)', 'Auto(AM-S9)',
       'Automatic (S10)'], dtype=object)

In [ ]:
data["class"].unique()

array(['Minicompact Cars', 'Two Seaters', 'Special Purpose Vehicle 2WD',
       'Special Purpose Vehicle 4WD', 'Subcompact Cars', 'Midsize Cars',
       'Midsize Station Wagons', 'Compact Cars',
       'Midsize-Large Station Wagons', 'Large Cars',
       'Small Station Wagons', 'Standard Pickup Trucks 2WD',
       'Vans, Passenger Type', 'Vans, Cargo Type',
       'Standard Pickup Trucks 4WD', 'Special Purpose Vehicles',
       'Small Pickup Trucks 2WD', 'Small Pickup Trucks 4WD', 'Vans',
       'Standard Pickup Trucks', 'Small Pickup Trucks', 'Vans Passenger',
       'Standard Pickup Trucks/2wd', 'Special Purpose Vehicles/2wd',
       'Special Purpose Vehicles/4wd', 'Sport Utility Vehicle - 4WD',
       'Sport Utility Vehicle - 2WD', 'Minivan - 2WD', 'Minivan - 4WD',
       'Special Purpose Vehicle', 'Small Sport Utility Vehicle 4WD',
       'Small Sport Utility Vehicle 2WD',
       'Standard Sport Utility Vehicle 4WD',
       'Standard Sport Utility Vehicle 2WD'], dtype=object)

In [ ]:
data["class"].value_counts()

class
Compact Cars                          5508
Subcompact Cars                       4872
Midsize Cars                          4395
Standard Pickup Trucks                2354
Sport Utility Vehicle - 4WD           2082
Large Cars                            1891
Two Seaters                           1886
Sport Utility Vehicle - 2WD           1627
Small Station Wagons                  1499
Special Purpose Vehicles              1455
Minicompact Cars                      1260
Standard Pickup Trucks 2WD            1177
Vans                                  1141
Standard Pickup Trucks 4WD             986
Midsize-Large Station Wagons           656
Special Purpose Vehicle 2WD            613
Small Pickup Trucks                    538
Small Sport Utility Vehicle 4WD        526
Midsize Station Wagons                 523
Vans, Cargo Type                       438
Small Pickup Trucks 2WD                436
Standard Sport Utility Vehicle 4WD     434
Small Sport Utility Vehicle 2WD        403
Miniv

In [ ]:
data["class"] = data["class"].replace('Vans Passenger', 'Vans, Passenger Type')
data["class"] = data["class"].replace('Standard Pickup Trucks/2wd', 'Standard Pickup Trucks 2WD')
data["class"] = data["class"].replace('Special Purpose Vehicles/2wd', 'Special Purpose Vehicle 2WD')
data["class"] = data["class"].replace('Special Purpose Vehicles/4wd', 'Special Purpose Vehicle 4WD')

In [ ]:
data["class"].value_counts()

class
Compact Cars                          5508
Subcompact Cars                       4872
Midsize Cars                          4395
Standard Pickup Trucks                2354
Sport Utility Vehicle - 4WD           2082
Large Cars                            1891
Two Seaters                           1886
Sport Utility Vehicle - 2WD           1627
Small Station Wagons                  1499
Special Purpose Vehicles              1455
Minicompact Cars                      1260
Standard Pickup Trucks 2WD            1181
Vans                                  1141
Standard Pickup Trucks 4WD             986
Midsize-Large Station Wagons           656
Special Purpose Vehicle 2WD            615
Small Pickup Trucks                    538
Small Sport Utility Vehicle 4WD        526
Midsize Station Wagons                 523
Vans, Cargo Type                       438
Small Pickup Trucks 2WD                436
Standard Sport Utility Vehicle 4WD     434
Small Sport Utility Vehicle 2WD        403
Miniv

In [ ]:
data.to_csv("fuel_cleaned.csv", index = False)